<a href="https://colab.research.google.com/github/MMR1318/Maheshreddy_INFO5731_Fall2024/blob/main/Mottakatla_Mahesh_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [2]:
!pip install selenium

In [3]:
from selenium import webdriver

In [4]:
!apt-get install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [6]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument ("--verbose")
  options.add_argument('-no-sandbox')
  options.add_argument('-headless')
  options.add_argument('--disable-gpu')
  options.add_argument ("--window-size=1920, 1200")
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver. Chrome (options=options)
  return driver

In [7]:
driver = web_driver()

In [8]:
from selenium import webdriver
!apt-get install -y chromium-driver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

def get_imdb_reviews(movie_ids, reviews_per_movie=400, max_retries=2, retry_delay=5):
    reviews_list = []
    buttons_list = []  # List to hold button information

    for movie_id in movie_ids:
        url = f"https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_ql_3"
        print(f"Scraping reviews for movie ID: {movie_id}")
        movie_reviews = []
        retries = 0

        while len(movie_reviews) < reviews_per_movie and retries < max_retries:
            try:
                driver.get(url)
                time.sleep(5)  # Wait for the page to load

                # Parse the page with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                # print(soup.prettify())  # Print the complete HTML of the page for inspection

                # Collect button information after parsing the page
                buttons = soup.find('button', id='load-more-trigger')
                print(buttons)

                # Load more reviews
                load_more_button = driver.find_element(By.ID, 'load-more-trigger')
                load_more_button.click()
                time.sleep(20)  # Wait for more reviews to load

                # Collect reviews after clicking the load more button
                reviews = soup.find_all('div', class_='text show-more__control')

                if not reviews:
                    retries += 1
                    time.sleep(retry_delay)
                    continue  # Retry if no reviews are found

                # Collect initial reviews
                for review in reviews:
                    if len(movie_reviews) >= reviews_per_movie:
                        break
                    movie_reviews.append(review.text.strip())

                retries = 0  # Reset retries after a successful attempt
                time.sleep(1)  # To avoid making too many requests quickly

            except Exception as e:
                print(f"An error occurred for movie ID: {movie_id}: {e}")
                retries += 1
                time.sleep(retry_delay)

        reviews_list.extend(movie_reviews)
        print(f"Collected {len(movie_reviews)} reviews for movie ID: {movie_id}")

    # Close the driver after scraping
    driver.quit()
    return reviews_list

def save_reviews_to_csv(reviews, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Review"])
        for review in reviews:
            writer.writerow([review])

if __name__ == "__main__":
    # List of IMDb movie IDs to collect reviews from
    movie_ids = [
        "tt15398776",  # Oppenheimer
        "tt13238346" # Past Lives
        ]

    reviews_per_movie = 500
    reviews = get_imdb_reviews(movie_ids, reviews_per_movie=reviews_per_movie)

    if reviews:
        save_reviews_to_csv(reviews, 'imdb_reviews.csv')
        print(f"Collected {len(reviews)} reviews and saved to imdb_reviews.csv")
    else:
        print("No reviews collected.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Scraping reviews for movie ID: tt15398776
<button class="ipl-load-more__button" data-target-container="reviews-container" id="load-more-trigger">Load More</button>
<button class="ipl-load-more__button" data-target-container="reviews-container" id="load-more-trigger">Load More</button>
<button class="ipl-load-more__button" data-target-container="reviews-container" id="load-more-trigger">Load More</button>
<button class="ipl-load-more__button" data-target-container="reviews-container" id="load-more-trigger">Load More</button>
<button class="ipl-load-more__button" data-target-container="reviews-container" id="load-more-trigger">Load More</button>
<button class="ipl-load-more__b

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [9]:
# Write code for each of the sub parts with proper comments.
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure nltk resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the reviews from the CSV file
def load_reviews_from_csv(filename):
    return pd.read_csv(filename)

# Clean text function
def clean_text(text):
    # (1) Remove noise, such as special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    stop_words = set(stopwords.words('english'))
    text_tokens = nltk.word_tokenize(text)
    text = ' '.join(word for word in text_tokens if word.lower() not in stop_words)

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    ps = PorterStemmer()
    text = ' '.join(ps.stem(word) for word in text.split())

    # (6) Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())

    return text

# Main function to process the reviews
def process_reviews(filename):
    df = load_reviews_from_csv(filename)

    # Apply the cleaning process
    df['Cleaned_Review'] = df['Review'].apply(clean_text)

    # Save the cleaned reviews to a new CSV file
    df.to_csv('cleaned_imdb_reviews.csv', index=False, encoding='utf-8')
    print(f"Cleaned reviews saved to 'cleaned_imdb_reviews.csv'.")

if __name__ == "__main__":
    process_reviews('imdb_reviews.csv')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cleaned reviews saved to 'cleaned_imdb_reviews.csv'.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [10]:
# Your code here
!pip install nltk
!pip install spacy

In [16]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree
from nltk import RegexpParser

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Load the cleaned reviews from the CSV file
def load_cleaned_reviews(filename):
    return pd.read_csv(filename)

# (1) Parts of Speech (POS) Tagging
def pos_tagging(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    # Count Nouns, Verbs, Adjectives, Adverbs
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}
    for word, tag in pos_tags:
        if tag.startswith('NN'):  # Noun
            pos_counts['Noun'] += 1
        elif tag.startswith('VB'):  # Verb
            pos_counts['Verb'] += 1
        elif tag.startswith('JJ'):  # Adjective
            pos_counts['Adjective'] += 1
        elif tag.startswith('RB'):  # Adverb
            pos_counts['Adverb'] += 1

    return pos_counts

# (2) Constituency Parsing and Dependency Parsing
def parse_sentence(sentence):
    # Constituency Parsing
    grammar = r"""
        NP: {<DT>?<JJ>*<NN.*>}  # Noun Phrase
        VP: {<VB.*><NP|PP|CLAUSE>+$}  # Verb Phrase
        PP: {<IN><NP>}  # Prepositional Phrase
        CLAUSE: {<NP><VP>}  # Clause
    """
    cp = RegexpParser(grammar)
    tree = cp.parse(nltk.pos_tag(word_tokenize(sentence)))
    print("Constituency Parse Tree:")
    print(tree)
    tree.pretty_print()

    # Dependency Parsing
    doc = nlp(sentence)
    print("\nDependency Parse Tree:")
    for token in doc:
        print(f"{token.text} --> {token.dep_} --> {token.head.text}")

# (3) Named Entity Recognition
def named_entity_recognition(text):
    doc = nlp(text)

    # Initialize an empty dictionary to count entities
    entities_count = {}

    for ent in doc.ents:
        # Increment the count for each entity label dynamically
        if ent.label_ in entities_count:
            entities_count[ent.label_] += 1
        else:
            entities_count[ent.label_] = 1

    return entities_count

# Main function to analyze the cleaned reviews
def analyze_reviews(filename):
    df = load_cleaned_reviews(filename)

    # Parts of Speech Tagging
    total_pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

    for review in df['Cleaned_Review']:
        pos_counts = pos_tagging(review)
        for key in total_pos_counts.keys():
            total_pos_counts[key] += pos_counts[key]

    print("Total POS Counts:")
    print(total_pos_counts)

    # Sentence for Parsing
    example_sentence = df['Cleaned_Review'].iloc[0]
    parse_sentence(example_sentence)

    # Named Entity Recognition
    total_entities = {'PERSON': 0, 'ORG': 0, 'GPE': 0, 'PRODUCT': 0, 'DATE': 0}
    for review in df['Cleaned_Review']:
        entities_count = named_entity_recognition(review)
        for key in total_entities.keys():
            # Check if the key exists in entities_count before adding
            if key in entities_count:
                total_entities[key] += entities_count[key]

    print("\nTotal Named Entities Count:")
    print(total_entities)

if __name__ == "__main__":
    analyze_reviews('/content/cleaned_imdb_reviews.csv')


Total POS Counts:
{'Noun': 68236, 'Verb': 19207, 'Adjective': 27741, 'Adverb': 7609}
Constituency Parse Tree:
(S
  one/CD
  (NP anticip/NN)
  (NP film/NN)
  (NP year/NN)
  (NP mani/NN)
  (NP peopl/NN)
  (NP includ/NN)
  (NP oppenheim/NN)
  larg/VBZ
  deliv/RB
  (NP much/JJ great/JJ feel/NN)
  (PP like/IN (NP love/NN))
  two/CD
  three/CD
  (NP hour/NN)
  (PP like/IN (NP hour/NN))
  (NP fact/NN)
  stop/VB
  (NP ador/NN)
  (NP entir/JJ thing/NN)
  know/VBP
  (NP christoph/NN)
  (NP nolan/NN)
  dunkirk/VBZ
  (NP click/JJ second/JJ watch/NN)
  (NP mayb/NN)
  oppenheim/VBD
  need/MD
  one/CD
  said/VBD
  (NP dont/NN)
  (NP feel/NN)
  need/VBP
  (NP rush/NN)
  see/VB
  soon/RB
  long/RB
  (NP exhaust/JJ filmbut/NN)
  (NP mani/JJ way/NN)
  (NP cant/JJ deni/NN)
  except/IN
  well/RB
  made/VBN
  one/CD
  (NP look/NN)
  (NP sound/NN)
  amaz/IN
  youd/JJ
  expect/VBP
  feel/VB
  (PP though/IN (NP accur/JJ captur/JJ time/NN))
  (NP period/NN)
  set/VBN
  (NP contain/NN)
  (NP amaz/JJ sound/NN)
  

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

https://myunt-my.sharepoint.com/:x:/r/personal/maheshreddymottakatla_my_unt_edu/Documents/cleaned_imdb_reviews.csv?d=wd2d91c2bfda4450b9251ad5ae25d416f&csf=1&web=1&e=8zCcLs

In [ ]:
# @title Default title text
## Code Organization and Readability

#I ensured that my code is clean and well-commented to make it easy for anyone reviewing it to understand the functionality. I also chose descriptive variable names to enhance readability.

## Handling Data Collection

#During the data collection process, I used both web scraping and API requests. I faced a few challenges with handling timeouts, but I implemented retries to ensure the data was collected reliably. I made sure to respect the site's terms and added a delay between requests to avoid overwhelming the server.

## Data Cleaning Approach

#I thoroughly cleaned the text data by removing irrelevant information like special characters, numbers, and stopwords. This step was essential to ensure accurate analysis. I also performed lemmatization to normalize the words, which helped in improving the syntactic analysis.

## Syntactic Analysis

#For syntactic analysis, I implemented POS tagging, constituency parsing, dependency parsing, and named entity recognition. One thing I found particularly interesting was how Named Entity Recognition (NER) helped in identifying key entities in the text, which could be useful for further analysis, such as extracting specific themes from the reviews.

## Challenges Faced

#One challenge I encountered was with handling large volumes of text data during cleaning and processing. To address this, I optimized my code by using vectorized operations where possible. Additionally, handling noisy data required some trial and error, especially when deciding on the best cleaning techniques for my dataset.

## Reflection

#Overall, I believe my approach to both data collection and analysis was effective, but there’s room for improvement, especially in expanding the data sources to get more diverse results. In future projects, I would like to explore more advanced NLP models to gain deeper insights from the text data.

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below
# Thoughts on the Assignment
# Challenges:

# Data Scraping Complexity: Initially, scraping data from IMDb using Selenium presented some challenges, particularly in handling dynamic content loading and ensuring that the necessary elements were available before interaction.
# Text Cleaning and Processing: Implementing various text cleaning techniques was intricate, especially ensuring that each step—such as stemming and lemmatization—was effectively applied without losing the essence of the reviews.
# Syntax and Structure Analysis: Conducting parts of speech tagging, parsing, and named entity recognition required a solid understanding of NLP concepts and the libraries used, which was both challenging and rewarding.
# Enjoyable Aspects:

# Learning New Tools: Using libraries like NLTK and SpaCy for text processing and analysis was exciting. They offer powerful tools for NLP that broadened my understanding of the field.
# Data Insights: Extracting meaningful insights from the cleaned data and conducting the analyses provided a sense of accomplishment. Seeing the results of the entity recognition and understanding the syntactic structure of sentences was particularly gratifying.
# Practical Application: Applying theoretical knowledge in a real-world scenario, such as analyzing movie reviews, made the assignment engaging and relevant.
# Time Management: The time provided to complete the assignment felt adequate. However, some tasks, such as troubleshooting the web scraping and ensuring the accuracy of the text analysis, required more time than initially anticipated. I appreciated having time to dive deeper into the aspects of NLP that I found intriguing.